In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import the data from the CSV file obtained in the prior SQL analysis step:
metrics = pd.read_csv('/content/drive/MyDrive/TravelTide/calculated metrics .csv')
metrics.head()

,user_id,hotel_hunter_index,average_bags_scaled,cancellation_rate_scaled,bargain_hunter_index,combined_booking_scaled,session_intensity_index
0,101961.0,NaN,0.080000,0.0,0.000058,1.0,0.176267
1,106907.0,NaN,1.000000,0.5,NaN,1.0,0.434310
2,181157.0,0.000495,0.200000,0.0,0.004943,1.0,0.292051
3,190866.0,NaN,0.080000,0.0,NaN,1.0,0.630760
4,204997.0,0.000898,0.066667,0.4,0.011774,0.8,0.161446


In [ ]:
# Checking formats of our data:
metrics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5998 entries, 0 to 5997
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   user_id                   5542 non-null   float64
 1   hotel_hunter_index        1646 non-null   float64
 2   average_bags_scaled       5178 non-null   float64
 3   cancellation_rate_scaled  5542 non-null   float64
 4   bargain_hunter_index      1718 non-null   float64
 5   combined_booking_scaled   5542 non-null   float64
 6   session_intensity_index   5998 non-null   float64
dtypes: float64(7)
memory usage: 328.1 KB


In [ ]:
# Step 1. Ranking all our metrics in descending order, keeping NaNs:
metrics['rank_hotel_hunter'] = metrics['hotel_hunter_index'].rank(na_option = 'keep', ascending=False)
metrics['rank_ave_bags'] = metrics['average_bags_scaled'].rank(na_option = 'keep', ascending=False)
metrics['rank_cancellation_rank'] = metrics['cancellation_rate_scaled'].rank(na_option = 'keep', ascending=False)
metrics['rank_bargain_hunter'] = metrics['bargain_hunter_index'].rank(na_option = 'keep', ascending=False)
metrics['rank_combined_booking'] = metrics['combined_booking_scaled'].rank(na_option = 'keep', ascending=False)
metrics['rank_session_activity'] = metrics['session_intensity_index'].rank(na_option = 'keep', ascending=False)
metrics.head(5)

,user_id,hotel_hunter_index,average_bags_scaled,cancellation_rate_scaled,bargain_hunter_index,combined_booking_scaled,session_intensity_index,rank_hotel_hunter,rank_ave_bags,rank_cancellation_rank,rank_bargain_hunter,rank_combined_booking,rank_session_activity
0,101961.0,NaN,0.080000,0.0,0.000058,1.0,0.176267,NaN,3380.0,3069.0,1679.0,1233.0,5988.0
1,106907.0,NaN,1.000000,0.5,NaN,1.0,0.434310,NaN,4.0,52.0,NaN,1233.0,2231.0
2,181157.0,0.000495,0.200000,0.0,0.004943,1.0,0.292051,827.0,913.5,3069.0,467.0,1233.0,5487.5
3,190866.0,NaN,0.080000,0.0,NaN,1.0,0.630760,NaN,3380.0,3069.0,NaN,1233.0,38.5
4,204997.0,0.000898,0.066667,0.4,0.011774,0.8,0.161446,619.0,3647.5,104.5,203.0,2675.5,5995.0


In [ ]:
# Step 2. Finding minimum rating for each user:
metrics['min'] = metrics[['rank_hotel_hunter',
                              'rank_ave_bags',
                              'rank_cancellation_rank',
                              'rank_bargain_hunter',
                              'rank_combined_booking',
                              'rank_session_activity']].min(axis=1)
metrics.head()

,user_id,hotel_hunter_index,average_bags_scaled,cancellation_rate_scaled,bargain_hunter_index,combined_booking_scaled,session_intensity_index,rank_hotel_hunter,rank_ave_bags,rank_cancellation_rank,rank_bargain_hunter,rank_combined_booking,rank_session_activity,min
0,101961.0,NaN,0.080000,0.0,0.000058,1.0,0.176267,NaN,3380.0,3069.0,1679.0,1233.0,5988.0,1233.0
1,106907.0,NaN,1.000000,0.5,NaN,1.0,0.434310,NaN,4.0,52.0,NaN,1233.0,2231.0,4.0
2,181157.0,0.000495,0.200000,0.0,0.004943,1.0,0.292051,827.0,913.5,3069.0,467.0,1233.0,5487.5,467.0
3,190866.0,NaN,0.080000,0.0,NaN,1.0,0.630760,NaN,3380.0,3069.0,NaN,1233.0,38.5,38.5
4,204997.0,0.000898,0.066667,0.4,0.011774,0.8,0.161446,619.0,3647.5,104.5,203.0,2675.5,5995.0,104.5


In [ ]:
# Step 3. Creating function for defining the most attractive perk for each user:
def perk_define(row):
    if row['rank_hotel_hunter'] == row['min']:
        perk = 'Free Hotel Meal'
    elif row['rank_ave_bags'] == row['min']:
        perk = 'Free Checked Bag'
    elif row['rank_cancellation_rank'] == row['min']:
        perk = 'No Cancellation Fee'
    elif row['rank_bargain_hunter'] == row['min']:
        perk = 'Exclusive Discount'
    elif row['rank_combined_booking'] == row['min']:
        perk = 'Night Free Hotel with Flight'
    elif row['rank_session_activity'] == row['min']:
        perk = 'Active Searcher'
    else:
        'Not Defined'
    return perk



In [ ]:
# Step 4: Utilizing our function to ascertain the rankings of customers:
metrics['most_attractive_perk'] = metrics.apply(perk_define, axis=1)
metrics.head()

,user_id,hotel_hunter_index,average_bags_scaled,cancellation_rate_scaled,bargain_hunter_index,combined_booking_scaled,session_intensity_index,rank_hotel_hunter,rank_ave_bags,rank_cancellation_rank,rank_bargain_hunter,rank_combined_booking,rank_session_activity,min,most_attractive_perk
0,101961.0,NaN,0.080000,0.0,0.000058,1.0,0.176267,NaN,3380.0,3069.0,1679.0,1233.0,5988.0,1233.0,Night Free Hotel with Flight
1,106907.0,NaN,1.000000,0.5,NaN,1.0,0.434310,NaN,4.0,52.0,NaN,1233.0,2231.0,4.0,Free Checked Bag
2,181157.0,0.000495,0.200000,0.0,0.004943,1.0,0.292051,827.0,913.5,3069.0,467.0,1233.0,5487.5,467.0,Exclusive Discount
3,190866.0,NaN,0.080000,0.0,NaN,1.0,0.630760,NaN,3380.0,3069.0,NaN,1233.0,38.5,38.5,Active Searcher
4,204997.0,0.000898,0.066667,0.4,0.011774,0.8,0.161446,619.0,3647.5,104.5,203.0,2675.5,5995.0,104.5,No Cancellation Fee


In [ ]:
# The number of users within each segment
metrics['most_attractive_perk'].value_counts()

Active Searcher                 1477
Free Checked Bag                1096
Free Hotel Meal                  955
Exclusive Discount               905
Night Free Hotel with Flight     807
No Cancellation Fee              758
Name: most_attractive_perk, dtype: int64

In [ ]:
# Downloading the file for further analysis in Tableau:
metrics.to_csv('/content/drive/MyDrive/TravelTide/Perks_data.csv')

In [ ]:
# Loading the data from csv file with users data:
users_data = pd.read_csv('/content/drive/MyDrive/TravelTide/users_data.csv')
users_data.head()

,user_id,birthdate,gender,married,has_children,home_country,home_city,home_airport,home_airport_lat,home_airport_lon,sign_up_date
0,623949,1976-05-21,M,True,False,usa,new york,LGA,40.777,-73.872,2023-02-27
1,385689,1974-10-08,F,False,True,usa,nashville,BNA,36.124,-86.678,2022-10-29
2,513241,1987-09-14,F,False,True,usa,las vegas,LAS,36.080,-115.152,2023-01-06
3,506990,1998-07-01,F,False,False,usa,lubbock,LBB,33.664,-101.823,2023-01-03
4,510707,1983-06-09,F,False,True,canada,toronto,YKZ,43.862,-79.370,2023-01-05


In [ ]:
users_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5998 entries, 0 to 5997
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   user_id           5998 non-null   int64  
 1   birthdate         5998 non-null   object 
 2   gender            5998 non-null   object 
 3   married           5998 non-null   bool   
 4   has_children      5998 non-null   bool   
 5   home_country      5998 non-null   object 
 6   home_city         5998 non-null   object 
 7   home_airport      5998 non-null   object 
 8   home_airport_lat  5998 non-null   float64
 9   home_airport_lon  5998 non-null   float64
 10  sign_up_date      5998 non-null   object 
dtypes: bool(2), float64(2), int64(1), object(6)
memory usage: 433.6+ KB


In [ ]:
# Convert birthdate to date format:
users_data['birthdate'] = pd.to_datetime(users_data['birthdate'], errors='coerce')

# Calculate age:
users_data['age'] = (pd.Timestamp.now() - users_data['birthdate'])//np.timedelta64(1,'Y')
users_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5998 entries, 0 to 5997
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   user_id           5998 non-null   int64         
 1   birthdate         5998 non-null   datetime64[ns]
 2   gender            5998 non-null   object        
 3   married           5998 non-null   bool          
 4   has_children      5998 non-null   bool          
 5   home_country      5998 non-null   object        
 6   home_city         5998 non-null   object        
 7   home_airport      5998 non-null   object        
 8   home_airport_lat  5998 non-null   float64       
 9   home_airport_lon  5998 non-null   float64       
 10  sign_up_date      5998 non-null   object        
 11  age               5998 non-null   int64         
dtypes: bool(2), datetime64[ns](1), float64(2), int64(2), object(5)
memory usage: 480.4+ KB


In [ ]:
# Cheking values in Gender column
users_data['gender'].value_counts()

F    5292
M     695
O      11
Name: gender, dtype: int64

In [ ]:
# Since amount of O-values is insignificant (only 0.18% of total) we will replace them by most frequent
# value ("F") and convert this column to boolean

replace_values = {'F' : True, 'O' : True, 'M' : False }
users_data['gender'] = users_data['gender'].map(replace_values)

In [ ]:
# Drop unnecessary columns from the 'users_data' DataFrame to keep only the relevant information.
users_data.drop(['birthdate','home_country',
                 'home_city', 'home_airport',
                 'home_airport_lat', 'home_airport_lon',
                 'sign_up_date'], axis='columns', inplace=True)

In [ ]:
users_data.head()

,user_id,gender,married,has_children,age
0,623949,False,True,False,47
1,385689,True,False,True,48
2,513241,True,False,True,36
3,506990,True,False,False,25
4,510707,True,False,True,40


In [ ]:
# Downloading the file for further analysis in Tableau:
users_data.to_csv('/content/drive/MyDrive/TravelTide/users_data_processed.csv')

In [ ]:
# Merge two datasets into a single dataset based on the user ID number:
data = users_data.merge(metrics, how='inner')
data.head()

,user_id,gender,married,has_children,age,hotel_hunter_index,average_bags_scaled,cancellation_rate_scaled,bargain_hunter_index,combined_booking_scaled,session_intensity_index,rank_hotel_hunter,rank_ave_bags,rank_cancellation_rank,rank_bargain_hunter,rank_combined_booking,rank_session_activity,min,most_attractive_perk
0,623949,False,True,False,47,NaN,0.133333,0.0,0.008929,1.00,0.383641,NaN,2063.0,3069.0,255.0,1233.0,3643.0,255.0,Exclusive Discount
1,385689,True,False,True,48,NaN,0.066667,0.0,0.001415,0.75,0.252304,NaN,3647.5,3069.0,995.0,2995.5,5812.5,995.0,Exclusive Discount
2,510707,True,False,True,40,NaN,0.200000,0.0,NaN,0.50,0.421659,NaN,913.5,3069.0,NaN,4343.5,2578.5,913.5,Free Checked Bag
3,520959,True,False,False,48,0.020384,0.000000,0.0,NaN,1.00,0.335253,28.0,4635.0,3069.0,NaN,1233.0,4837.5,28.0,Free Hotel Meal
4,527321,True,False,False,17,NaN,NaN,0.0,NaN,0.00,0.248848,NaN,NaN,3069.0,NaN,5283.5,5830.0,3069.0,No Cancellation Fee


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5542 entries, 0 to 5541
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   user_id                   5542 non-null   int64  
 1   gender                    5542 non-null   bool   
 2   married                   5542 non-null   bool   
 3   has_children              5542 non-null   bool   
 4   age                       5542 non-null   int64  
 5   hotel_hunter_index        1646 non-null   float64
 6   average_bags_scaled       5178 non-null   float64
 7   cancellation_rate_scaled  5542 non-null   float64
 8   bargain_hunter_index      1718 non-null   float64
 9   combined_booking_scaled   5542 non-null   float64
 10  session_intensity_index   5542 non-null   float64
 11  rank_hotel_hunter         1646 non-null   float64
 12  rank_ave_bags             5178 non-null   float64
 13  rank_cancellation_rank    5542 non-null   float64
 14  rank_bar

In [ ]:
# Downloading the file for further analysis in Tableau:
data.to_csv('/content/drive/MyDrive/TravelTide/user_id and segment label.csv')